# Lyst Scraper: a quick tutorial on web scraping

This notebook is a quick tutorial on how to scrap a website. I am not a web scraping expert at all, I am mainly still a beginner, so this tutorail is aimed at beginners mostly. However, I tried not to focus only on the __code__ but also on other aspects of web scraping. You can find plenty of complete tutorials for the use of Beautiful Soup or Selenium online. Instead, I wanted to show a glimpse of how to scrap a website, some useful tricks I learnt during my work.

# Part 1: Web scraping ?

Prior to start coding, it is important to understand what is a web scraper, how and __when__ to use it. More important, web scraping has a tricky __legal__ aspect, so be sure to read these few lines before you start playing with data.

## What is a web scraper ?

A web scraper is a program that __extracts__ data from the World Wide Web. In all my examples, I use web scraper to download images to create databases, but I can be used to extract pretty much any type of data you can find online.
Web scraping is different from __web crawling__, which is basically gathering and indexing every byte of info you can find on a website. As a crawler visits all the links on a page, I still use the term __crawl__ to describe that a scraper is visiting a link.

## Why would you scrap a website ?

There are many reasons to scrap a website. Personnally, I started to scrap to create databases for my __machine learning__ projects. But as I got into web scraping, I enjoyed it so much that I started to scrap some websites for fun (like Google Trends or op.gg for instance).

## The big question: is web scraping legal ?

Wellll... This is a good one. I am no lawyer, so I can't give you a detailed answer here, but here's what I understood after spending some time looking for a definitive answer. There is no __definitive__ answer. It heavily depends on what you do with the data you scrap. Here's a good blog about the subject : https://benbernardblog.com/web-scraping-and-crawling-are-perfectly-legal-right/. But always keep in mind that the fact that the data is publicly accesible __does not mean you have the right to sracp it and use it for your own projects__. Simples advices are:
-  If you're not sure, ask the owner of the website for his __permisssion__ to scrap it
-  If you think the data might be sensible, __ask a specialized lawyer__
-  Follow the rules specified in the __robots.txt__ file. Every website has a basic HTML page describing its policy about web scraping. You can find it at _http://www.website.com/robots.txt_. You can have nice surprises, as some websites allow web scraping to an extend (ex: op.gg for League of Legends statistics).
-  Be __polite__. I will insist a lot on this during this tutorial, but servers are not made to handle 10 requests a second per user. This is not a normal behaviour for a human being, and it can seriously harm a website. If you're taking their data - especially without permission - at least do it right.
   

# Part 2: "Naive" web scraping

This "Naive" part is very important. I call it naive because it requires some manual work to understand how to retrieve the data from a website, but this step is essential to understand how a scraper works, especially if - like me - you don't have much knowledge on webprograming.

Keep in mind that this notebook is not designed to be a complete all-in-one tutorial for beginners. You can find plenty of tutorials for Beautiful Soup or the Selenium syntax on the web, so I won't detail each ligne of code I wrote. This notebook is written by a beginner for beginners, it is here to give a glimpse on how to scrap a website, and some useful tips I found by myself.

During this tutorial, we will try to get images from _lyst.fr_, an online fashion shopping website.

First, let's import the useful librairies we're gonna use during our project. They are all straight-forward to install using __pip__, except Selenium which needs you to install a bit more but you can find everything on the documentation of Selenium online.
During this tutorial, I use Beautiful Soup, a Python librairy for extracting data from HTML files. Here's the link to the documentation https://www.crummy.com/software/BeautifulSoup/bs4/doc/.
Beautiful Soup is a very useful and complete tool for basic webscraping, however some more complex tasks cannot be achieved. For example, _lyst.fr_ features an __infinite scroll__ which requires us to use an automated browser using Selenium (Documentation at https://www.seleniumhq.org/ ). Tip for installing: go to https://github.com/mozilla/geckodriver/releases and get the geckodriver corresponding to your Mozilla Browser if you use Mozilla.


In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import sys
import os
import traceback
import re
import time
import json
import unidecode
import shutil

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

When you scrap a website using Selenium or Beautiful, __your__ computer is accesing the data. It seems dumb to state this, but remember that a website expects human user to access its data, not automated bots. That's why we need to disguise our bot to make him look like a normal human user (_spoiler: we won't succeed_).

Headers are the __fingerprint__ of your browser. I always specify my true headers (in my case, Firefox Headers) when I scrap a website, because it doesn't give much info on myself, and it seems more realistic. You can find your headers using this website https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending, look for the __USER_AGENT__ info.

We're gonna try with one headers first. If this doesn't work, we can try using proxy rotation and multi-headers to hide our identity. But given the fact that we are not going to scrap a lot of sensitive data, we might be able to go anonymous. I've been nlocked from several websites beacause I was careless before so we should be careful. Anyway, when you crawl a website, remember that your request are not normal behaviour from a human user. You may do 10 request/second, which is way too much for a human user and might harm the website server. Please, be polite and use some sleep to relieve the servers.

In [2]:
# Specify your web browser's headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'}

# Specify the location to save the data
DownloadPath = "/media/arthur/DATA/Databases/LystScrapingHomme"

# We create a dictionary with all cat. and subcat. of the Lyst catalogue
words_to_search =  {
    'shirts':['casual+shirts','formal+shirts'],
    'suits':['2+piece+suits', '3+piece+suits','evening+suits'],
    'jeans':['bootcut+jeans', 'straight+jeans', 'relaxed+jeans', 'tapered+jeans', 'skinny+jeans', 'slim+jeans'],
    'coats':['trench+coats', 'short+coats', 'long+coats',  'parka+coats'],
    'pants':['casual+pants', 'formal+pants'],
    'beachwear':['trunks', 'boardshorts'],
    'knitwear':['cardigans','v+neck', 'crew+neck', 'turtlenecks', 'zipped+sweaters', 'sleeveless+sweaters'],
    'shorts':['bermudas+shorts', 'formal+shorts', 'casual+shorts','cargo+shorts'],
    'underwear':['boxers','socks', 'undershirt','briefs'],
    'sweats':['sweatpants', 'sweatshorts','tracksuits','sweatshirts', 'hoodies'],
    't-shirts':['polo+shirts','sleeveless+t-shirts','short+sleeve+t-shirts','long+sleeve+t-shirts'],
    'jackets':['formal+jackets', 'leather+jackets','waistcoast','casual+jackets','parka+jackets'],
    'nightwear':['pyjamas', 'robes']
}

url_download_base = "https://cdna.lystit.com/520/650/n/"

# Number of scrolls (because of infinite scroll)
number_of_scrolls = 200

First, we're gonna do a little test on just the 'Shirts' category. Our goal is to download all the images from this category and to save them locally on our computer. Selenium is gonna create an instance of our web browser (in my case, Mozilla Firefox) which will be controlled by this script. Be careful which retriever you use for the image, for example I first tried urllib.urlretrieve, but I got blocked by the website. If that happens, try another retriever (ex here, requests) or build your own.

In [52]:
to_scrap = words_to_search['shirts']
    
# Use selenium to create a firefox instance
driver = webdriver.Firefox()
extensions = {"jpg", "jpeg", "png"}

# Loop on all the clothes
for clothe in to_scrap:
    url = "https://www.lyst.fr/parcourir/vetements-pour-homme/?category=" + 'shirts' +  "&subcategory=" + clothe
    driver.get(url)
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        
        # If the website is a bit buggy, be smart
        time.sleep(0.7)
        driver.execute_script("window.scrollBy(0, -300)")
        
        # Don't forget this sleep to let the image appear on the page
        time.sleep(1.2)
        
    # Now we download all the images. BE POLITE AND DON'T GET BAN.
    images = driver.find_elements_by_xpath('//div[contains(@class, "product-card__image")]')
    print(len(images))
    i = 0
    for img in images:
        time.sleep(0.1)
        i+=1
        # Find the image and its src
        image = img.find_element_by_tag_name("img")
        print("Dowloading image no", i)
        
        # We modify the src to access the full size image
        try:
            image_url = url_download_base + image.get_attribute("src")[35::]
        except:
            print("Source not found.")
            continue
        print(image_url)
        image_name = image_url.split("/")[-1]
        file_name =DownloadPath + '/shirts/' + clothe + "/" + image_name
        try:
            r = requests.get(image_url, stream=True, headers=headers)
            f = open(file_name, 'wb')
            shutil.copyfileobj(r.raw, f)
        except:
            print("Error while downloading the image.")
            continue
        

    print("Downloading over.")


3449
Dowloading image no 1
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/3d053cf8/acne-black-Chemise.jpeg
Dowloading image no 2
https://cdna.lystit.com/520/650/n/photos/yoox/d75ee014/henry-cottons-blue-Chemise.jpeg
Dowloading image no 3
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/912755eb/jil-sander-Beige-Chemise.jpeg
Dowloading image no 4
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/a6ccde68/marni-pink-Chemise.jpeg
Dowloading image no 5
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/a70e2478/apc-blue-Caban-en-laine.jpeg
Dowloading image no 6
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/aa2ccea5/louis-vuitton-purple-Chemise.jpeg
Dowloading image no 7
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/0de35277/louis-vuitton-black-Veste.jpeg
Dowloading image no 8
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/975fcf66/burberry-Beige-Chemise.jpeg
Dowloading image no 9
https://cdna.lystit.

Dowloading image no 104
Source not found.
Dowloading image no 105
Source not found.
Dowloading image no 106
Source not found.
Dowloading image no 107
Source not found.
Dowloading image no 108
Source not found.
Dowloading image no 109
https://cdna.lystit.com/520/650/n/photos/yoox/cad221ec/levis-Bleu-fonce-Chemise.jpeg
Dowloading image no 110
https://cdna.lystit.com/520/650/n/photos/matchesfashion/a82d0d68/givenchy-grey-Chemise-en-denim-delave-a-col-en-pointe.jpeg
Dowloading image no 111
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/27d204c2/dries-van-noten-purple-Chemise.jpeg
Dowloading image no 112
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/0a8bbca9/acne-grey-Chemise.jpeg
Dowloading image no 113
https://cdna.lystit.com/520/650/n/photos/yoox/15a57360/fendi-black-Chemise.jpeg
Dowloading image no 114
https://cdna.lystit.com/520/650/n/photos/matchesfashion/cfe67c27/balenciaga-grey-Chemise-en-coton-double-epaisseur.jpeg
Dowloading image no 115
https://cdna.l

Dowloading image no 198
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/dd1c418b/dries-van-noten-grey-Chemise.jpeg
Dowloading image no 199
https://cdna.lystit.com/520/650/n/photos/stylebop/2d377a6b/valentino--Chemise-en-coton-a-broderie.jpeg
Dowloading image no 200
https://cdna.lystit.com/520/650/n/photos/stylebop/547c50f4/golden-goose-deluxe-brand--Chemise-en-coton-melange-avec-patchs.jpeg
Dowloading image no 201
https://cdna.lystit.com/520/650/n/photos/stylebop/c955b727/oamc--Chemise-en-lin-et-en-coton-imprimes.jpeg
Dowloading image no 202
https://cdna.lystit.com/520/650/n/photos/stylebop/db5820af/valentino--Chemise-camouflage-en-coton-avec-medaille.jpeg
Dowloading image no 203
https://cdna.lystit.com/520/650/n/photos/stylebop/ab257fb6/acne--Chemise-en-coton-Ohio-Face.jpeg
Dowloading image no 204
https://cdna.lystit.com/520/650/n/photos/stylebop/573d5e7d/jil-sander--Chemise-en-coton-Pigmento.jpeg
Dowloading image no 205
Source not found.
Dowloading image no 206
Source no

Dowloading image no 269
https://cdna.lystit.com/520/650/n/photos/farfetch/edd0fe10/gucci-blue-Chemise-Coco-Capitan-a-rayures.jpeg
Dowloading image no 270
https://cdna.lystit.com/520/650/n/photos/farfetch/3add6d9c/burberry-blue-Chemise-imprimee.jpeg
Dowloading image no 271
https://cdna.lystit.com/520/650/n/photos/farfetch/5809c722/ports-1961-blue-Chemise-a-manches-courtes.jpeg
Dowloading image no 272
https://cdna.lystit.com/520/650/n/photos/farfetch/f6d4bc52/haider-ackermann-white-Chemise-a-motif-de-pois.jpeg
Dowloading image no 273
https://cdna.lystit.com/520/650/n/photos/farfetch/94ca5b39/neighborhood-blue-Chemise-a-poches-poitrine-en-denim.jpeg
Dowloading image no 274
https://cdna.lystit.com/520/650/n/photos/farfetch/0e4c396e/craig-green-grey-Chemise-rayee-a-capuche.jpeg
Dowloading image no 275
Source not found.
Dowloading image no 276
Source not found.
Dowloading image no 277
Source not found.
Dowloading image no 278
Source not found.
Dowloading image no 279
Source not found.
Dowloa

Dowloading image no 340
Source not found.
Dowloading image no 341
Source not found.
Dowloading image no 342
Source not found.
Dowloading image no 343
Source not found.
Dowloading image no 344
Source not found.
Dowloading image no 345
Source not found.
Dowloading image no 346
Source not found.
Dowloading image no 347
Source not found.
Dowloading image no 348
Source not found.
Dowloading image no 349
https://cdna.lystit.com/520/650/n/photos/ssense/2e65c703/ps-by-paul-smith-navy-Chemise-bleu-marine-Pin-Dot-Casual.jpeg
Dowloading image no 350
https://cdna.lystit.com/520/650/n/photos/ssense/9a537ed4/comme-des-garcons-black-Chemise-asymetrique-noire.jpeg
Dowloading image no 351
https://cdna.lystit.com/520/650/n/photos/ssense/d2f3e1b7/rag-bone--Chemise-a-manches-courtes-noire-Smith.jpeg
Dowloading image no 352
https://cdna.lystit.com/520/650/n/photos/ssense/31883c8d/dsquared--Chemise-fleurie-beige-Western.jpeg
Dowloading image no 353
https://cdna.lystit.com/520/650/n/photos/ssense/4b3effd6/ba

...

Dowloading image no 3286
https://cdna.lystit.com/520/650/n/photos/asos/96971092/noak-white-Chemise-habille-manches-longues.jpeg
Dowloading image no 3287
https://cdna.lystit.com/520/650/n/photos/asos/a21d8960/asos-navy-Chemise-ajuste.jpeg
Dowloading image no 3288
https://cdna.lystit.com/520/650/n/photos/yoox/d9f0782f/mcq-red-Chemise.jpeg
Dowloading image no 3289
https://cdna.lystit.com/520/650/n/photos/yoox/01f6c546/etro-Vert-militaire-Polo.jpeg
Dowloading image no 3290
Source not found.
Dowloading image no 3291
Source not found.
Dowloading image no 3292
Source not found.
Dowloading image no 3293
Source not found.
Dowloading image no 3294
Source not found.
Dowloading image no 3295
Source not found.
Dowloading image no 3296
Source not found.
Dowloading image no 3297
Source not found.
Dowloading image no 3298
Source not found.
Dowloading image no 3299
Source not found.
Dowloading image no 3300
Source not found.
Dowloading image no 3301
Source not found.
Dowloading image no 3302
Source not

Dowloading image no 3373
https://cdna.lystit.com/520/650/n/photos/h%26m/0ddf3be8/hm-blue-Chemise-a-col-mao.jpeg
Dowloading image no 3374
Source not found.
Dowloading image no 3375
Source not found.
Dowloading image no 3376
https://cdna.lystit.com/520/650/n/photos/h%26m/108ff4ad/hm-black-Chemise-stretch-Slim-fit.jpeg
Dowloading image no 3377
https://cdna.lystit.com/520/650/n/photos/h%26m/36101b27/hm-black-Chemise-stretch-Slim-fit.jpeg
Dowloading image no 3378
https://cdna.lystit.com/520/650/n/photos/h%26m/a2d91328/hm-blue-Chemise-Regular-fit.jpeg
Dowloading image no 3379
https://cdna.lystit.com/520/650/n/photos/h%26m/b007352a/hm-blue-Chemise-a-motif-Slim-fit.jpeg
Dowloading image no 3380
https://cdna.lystit.com/520/650/n/photos/h%26m/bdd1c732/hm-Jaune-Chemise-en-lyocell-Regular-fit.jpeg
Dowloading image no 3381
https://cdna.lystit.com/520/650/n/photos/h%26m/293a481f/hm-blue-Chemise-Regular-fit.jpeg
Dowloading image no 3382
https://cdna.lystit.com/520/650/n/photos/h%26m/e5d1d112/hm-Beige

271
Dowloading image no 1
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/c2d41790/comptoir-gl-white-Chemise-Lerca.jpeg
Dowloading image no 2
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/e7e9f9d7/zadig-and-voltaire-red-Chemise.jpeg
Dowloading image no 3
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/cc567579/comptoir-gl-Bleu-ciel-Chemise-Mitro-straight-fit.jpeg
Dowloading image no 4
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/d715f140/acne-Marine-Veste.jpeg
Dowloading image no 5
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/b0244d2f/mise-au-green-white-Chemise-Atin.jpeg
Dowloading image no 6
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/913df45a/galeries-lafayette-white-Chemise-Mtim-slim-fit.jpeg
Dowloading image no 7
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/b0988913/arrow-Bleu-ciel-Chemise-en-coton-Slim-Fit.jpeg
Dowloading image no 8
https://cdna.lystit.com/520/650/n/photos/galerieslafa

Dowloading image no 99
Source not found.
Dowloading image no 100
Source not found.
Dowloading image no 101
Source not found.
Dowloading image no 102
Source not found.
Dowloading image no 103
Source not found.
Dowloading image no 104
Source not found.
Dowloading image no 105
Source not found.
Dowloading image no 106
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/a6b6873a/sisley-Rouge-fonce-Chemise-en-coton-a-micro-motif-col-officier-slim-fit.jpeg
Dowloading image no 107
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/9d7debf5/fil-a-fil-Bleu-ciel-Chemise-en-twill-de-coton-a-micro-rayures-slim-fit.jpeg
Dowloading image no 108
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/10392bcf/comptoir-gl-Blanc-et-bleu-Chemise-Lancon.jpeg
Dowloading image no 109
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/fe3fa85f/el-ganso-white-Chemise-unie-en-coton.jpeg
Dowloading image no 110
https://cdna.lystit.com/520/650/n/photos/galerieslafayette/283ed109/fil-a-

Dowloading image no 189
https://cdna.lystit.com/520/650/n/photos/matchesfashion/396c85e9/saint-laurent-blue-Chemise-a-motif-tartan-et-bords-francs.jpeg
Dowloading image no 190
https://cdna.lystit.com/520/650/n/photos/matchesfashion/fd77e77a/prada-blue-Chemise-en-velours-cotele-a-manchettes-simples.jpeg
Dowloading image no 191
https://cdna.lystit.com/520/650/n/photos/matchesfashion/b6b78080/saturdays-surf-nyc-white-Chemise-a-col-boutonne-Crosby-Oxford.jpeg
Dowloading image no 192
https://cdna.lystit.com/520/650/n/photos/matchesfashion/fe0ee1cb/officine-generale-white-Chemise-en-coton-fil-coupe-Gaspard.jpeg
Dowloading image no 193
https://cdna.lystit.com/520/650/n/photos/yoox/a61c82fb/billtornade-black-Chemise.jpeg
Dowloading image no 194
https://cdna.lystit.com/520/650/n/photos/matchesfashion/0f9ed865/alexander-mcqueen-black-Chemise-a-col-Dancing-Skeleton.jpeg
Dowloading image no 195
https://cdna.lystit.com/520/650/n/photos/matchesfashion/dc4e5151/srensen-blue-Chemise-en-coton-Officer.j

This was a "basic" scraper: it only downloaded the images, saving them where we wanted (plus a lot of images can't be downloaded because the source is not specified). Now, we need all the info about the product: retailer, brand, description, save the url... Let's do a smarter scraper, not losing all this.

First, we try to get all the data from a single url and save it to a nice dictionary.

In [40]:
url = 'https://www.lyst.fr/vetements/etoile-isabel-marant-pre-owned-chemise-en-lin/'
html = requests.get(url, headers=headers)
soup = BeautifulSoup(html.text, 'lxml')
dico = {}

# Name of the product
name_brand = soup.find('div', {'itemprop':'brand'}).contents[1].string.replace("\n", "")
dico['brand'] = name_brand
print(name_brand)

# Short description
short_description = soup.find('div', {'itemprop':'name'}).contents[0].string.replace("\n", "")
dico['short-description'] = short_description
print(short_description)

# Retailer 
retailer = soup.find('a', {'reason':'retailer-link'}).contents[1].string[19::].replace("\n", "")
dico['retailer'] = retailer
print(retailer)

# Long description
long_description = soup.find('div', {'class':'product-description__details text-paragraph mb0'}).contents[1].string.replace("\n", "")
dico['long-description'] = long_description
print(long_description)

# Url of the main image
url_main_img = soup.find('img', {'class':'image-gallery-main-img'})['src'].replace("\n", "")
dico['url-main-image'] = url_main_img
print(url_main_img)

# Url of thumbnail images (if they exist)
other_imgs = soup.find('div', {'is':'gallery-thumbnails'}).contents

# Other_imgs is a list of length: 2*NB_OF_OTHER_IMGS+1
nb_imgs = int(len(other_imgs)/2)
dico['other-images-url'] = []
for i in range(1, nb_imgs):
    print(other_imgs[2*i+1]['href'])
    dico['other-images-url'].append(other_imgs[2*i+1]['href'])
    
print(dico)


Étoile Isabel Marant
Chemise en lin homme
Vestiaire Collective
Chemise à carreaux, boutonnée sur le devant, patte de boutonnage invisible, manches longues avec boutons aux poignets, deux poches plaquées sur la poitrine, bords arrondis. Composition : 60% lin, 40% coton.
https://cdnd.lystit.com/520/650/n/photos/vestiairecollective/1d20006e/etoile-isabel-marant-Ecru-Chemise-en-lin.jpeg
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/27ef8715/etoile-isabel-marant-Ecru-Chemise-en-lin.jpeg
https://cdna.lystit.com/520/650/n/photos/vestiairecollective/fad7a72e/etoile-isabel-marant-Ecru-Chemise-en-lin.jpeg
{'other-images-url': ['https://cdna.lystit.com/520/650/n/photos/vestiairecollective/27ef8715/etoile-isabel-marant-Ecru-Chemise-en-lin.jpeg', 'https://cdna.lystit.com/520/650/n/photos/vestiairecollective/fad7a72e/etoile-isabel-marant-Ecru-Chemise-en-lin.jpeg'], 'long-description': 'Chemise à carreaux, boutonnée sur le devant, patte de boutonnage invisible, manches longues avec bou

Now that we have a Beautiful Soup instance that gets all of this data, we can test it on a full category because writing the final crawler.

In [ ]:
# Same start as before, we need to reach the bottom 
to_scrap = words_to_search['shirts']
Base = 'https://www.lyst.fr'
    
# Use selenium to create a firefox instance
driver = webdriver.Firefox()
extensions = {"jpg", "jpeg", "png"}

# Create a list to save the data in json format
dico = []


# Loop on all the clothes
for clothe in to_scrap:
    print("Scrolling to the bottom of the page for category", clothe, "...")

    url = "https://www.lyst.fr/parcourir/vetements-pour-homme/?category=" + 'shirts' +  "&subcategory=" + clothe
    driver.get(url)
    for _ in range(200):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        
        # If the website is a bit buggy, be smart
        time.sleep(0.7)
        driver.execute_script("window.scrollBy(0, -300)")
        
        # Don't forget this sleep to let the image appear on the page
        time.sleep(1.2)
    print("Done !")
    
    # Now, we're gonna use the Lyst page of the product
    url_products_driver = driver.find_elements_by_xpath('//div[contains(@class, "product-card__details")]')
    print("Products scraped for category", clothe, ":", len(url_products_driver))
    for ind, product in enumerate(url_products_driver):
        time.sleep(0.1)
        print("Extracting data for product no", ind, "/", len(url_products_driver))
        dico_product = {}
        prod = product.find_element_by_tag_name("a")
        # Get the url of the product page
        url_prod = prod.get_attribute("href")
        
        # We use Beautiful Soup now, cause it is simpler to use and doesn't open a browser page
        html = requests.get(url_prod, headers = headers)
        soup = BeautifulSoup(html.text, 'lxml')
        name_brand = soup.find('div', {'itemprop':'brand'}).contents[1].string.replace("\n", "")
        short_description = soup.find('div', {'itemprop':'name'}).contents[0].string.replace("\n", "")
        retailer = soup.find('a', {'reason':'retailer-link'}).contents[1].string[19::].replace("\n", "")
        long_description = soup.find('div', {'class':'product-description__details text-paragraph mb0'}).contents[1].string.replace("\n", "")
        
        # Download the main image with the name of the retailer in its name
        url_main_img = soup.find('img', {'class':'image-gallery-main-img'})['src'].replace("\n", "")
        image_name = url_main_img.split("/")[-1]
        file_name =DownloadPath + '/shirts/' + clothe + "/" + retailer.replace(" ", "_") + "_" + image_name
        try:
            r = requests.get(url_main_img, stream=True, headers=headers)
            f = open(file_name, 'wb')
            shutil.copyfileobj(r.raw, f)
        except:
            print("Error while downloading the image")
            continue

        
        # Download the thumbnail image also with the name of the retailer in their name
        other_imgs = soup.find('div', {'is':'gallery-thumbnails'})
        dico_product['other-images-url'] = []
        
        # Not all images have thumbnails images
        if other_imgs is not None:
            other_imgs = soup.find('div', {'is':'gallery-thumbnails'}).contents
            nb_imgs = int(len(other_imgs)/2)
            for i in range(1, nb_imgs):
                dico_product['other-images-url'].append(other_imgs[2*i+1]['href'])
                image_name = other_imgs[2*i+1]['href'].split("/")[-1]
                file_name =DownloadPath + '/shirts/' + clothe + "/" + retailer.replace(" ", "_") + "_" + image_name
                try:
                    r = requests.get(other_imgs[2*i+1]['href'].replace("\n", ""), stream=True, headers=headers)
                    f = open(file_name, 'wb')
                    shutil.copyfileobj(r.raw, f)
                except:
                    print("Error while downloading the image.")
                    continue
        dico_product['brand'] = name_brand
        dico_product['short-description'] = short_description
        dico_product['retailer'] = retailer
        dico_product['long-description'] = long_description
        dico_product['url-main-image'] = url_main_img
        
        # Put the dico of the product in the list of all data
        dico.append(dico_product)
        
# Save all to  a json file
with open("shirts.json", "w") as fout:
    json.dump(dico, fout)
        

Scrolling to the bottom of the page for category casual+shirts ...


Now we can write our "brutal" scraper that crawls all the subcategories we chose. This methods works, but has several flaws: 
-  it is __long__. Selenium's infinite scroll needs the image to load each time we crawl, plus we had to be a little bit tricky by scrolling up to de-bug the website. This is not a major concern if you have time, but it is always good to go fast when processing web scraping.
-   you can get __blocked__ by the admin of the website's servers. You ask for many requests in a quite short amount of time, so you are likely to get caught.
-  to scrap the website this way, a lot of __manual__ work has to be done to understand how the website is built. This takes some time, and can be tricky if the website is obscure or complex.

# Part 2: Using the architecture of a website